In [ ]:
import os
os.chdir('..')

In [ ]:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt

import Yukawa_SINDy as ys
import cross_validation as cv

from importlib import reload
import pickle as pkl
with open('scaling_const.float', 'rb') as f:
    SCALING_CONST = pkl.load(f)

In [ ]:
scatter_kwargs = {}
scatter_kwargs["facecolors"] = 'none'
scatter_kwargs["edgecolors"] = 'tab:blue'

In [1]:
def plot_dev_vs_error(coefficients, prediction_errors):
    coef_devs = cv.compute_coef_devs(coefficients)
    fig, ax = plt.subplots()
    ax.scatter(coef_devs, prediction_errors, **scatter_kwargs)

    # labels
    ax.set_title("Prediction error as a model selection criterion")
    ax.set_xlabel("Coef. dev.")
    ax.set_ylabel("Pred. error")

    return fig, ax

# need to move compute_coef_devs somewhere